In [1]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm import tqdm
import talib

In [2]:
# 这里导入数据
df = pd.read_csv("../数据获取/聚宽数据/最新数据.csv")
df.columns = ['id','i', 'Date','Open','High','Low','Close','Volume']
df['id2'] =df['id'].shift()
df['i'] = range(len(df))
ids = list(df.loc[df['id'] != df['id2'], 'i']) # 这个是单个股票序号的起始。
ids.append(len(df)-1) # 添加最后1天。
df.head()

,id,i,Date,Open,High,Low,Close,Volume,id2
0,000001.XSHE,0,2018-12-24,8.91,8.96,8.83,8.93,53694517.0,NaN
1,000001.XSHE,1,2018-12-25,8.81,8.94,8.73,8.86,61867885.0,000001.XSHE
2,000001.XSHE,2,2018-12-26,8.87,8.93,8.79,8.82,44415942.0,000001.XSHE
3,000001.XSHE,3,2018-12-27,8.96,9.00,8.80,8.80,65873247.0,000001.XSHE
4,000001.XSHE,4,2018-12-28,8.83,8.97,8.83,8.89,60812019.0,000001.XSHE


In [39]:
after_up_max_1 = [] # 涨停后一天的情况
after_up_max_2 = [] # 第二天的情况
with tqdm(total=len(set(df['id']))) as pbar:
    for j in range(len(ids)-1):
        # 对每一个股票进行遍历
        df2 = df.iloc[ids[j]:ids[j+1],:].copy() # 这里筛选一只股票。
        df2['Close2'] = df2['Close'].shift(1) # 收盘价移动到下一天
        df2['Close3'] = df2['Close'].shift(2)
        # 涨停的判断，我这里是95%以上
        # for i in ((df2['Close']-df2['Open'])/df2['Open']*100).tolist():
        #     print(i)
        df2['up_max'] = 0 
        df2.loc[(df2['Close']-df2['Close2'])/df2['Close2'] > 0.098, 'up_max'] = 1 # 这一天的是涨停的。
        df2['up_max2'] = df2['up_max'].shift(1) # 下一天
        df2['up_max3'] = df2['up_max'].shift(2) # 下一天
        # 这里要取消开盘价收盘价最高价最低价是一样的情况，这种，我是上不去的。
        df2.loc[df2['High'] == df2['Low'], 'up_max'] = 0
        
        # 然后这里计算涨停后一天的情况。
        # 我这里单纯比较收盘价。
        # df2.dropna(axis=0, how='any',inplace=True) # 先删除无效的
        # 这里计算涨幅
        df3 = df2.loc[df2['up_max2'] == 1, :].copy()
        after_up_max_1.extend(((df3['Close']-df3['Close2'])/df3['Close2']*100).tolist())
        #
        df4 = df2.loc[df2['up_max3'] == 1, :].copy()
        after_up_max_2.extend(((df3['Close']-df3['Close3'])/df3['Close3']*100).tolist())
        pbar.update(1)

        

100%|██████████████████████████████████████████████████████████████████████████████| 4275/4275 [00:54<00:00, 78.09it/s]


In [40]:
len(after_up_max_1)

60538

In [41]:
sum(after_up_max_1)/len(after_up_max_1)

1.9100969963374856

In [42]:
len([i for i in after_up_max_1 if i > 0])/len(after_up_max_1)*100

58.17503055931812

可以看到涨停后的下一天还是涨的，不过概率不大。

In [44]:
# 第二天的
sum(after_up_max_2)/len(after_up_max_2)

12.852112549117152

In [45]:
len([i for i in after_up_max_2 if i > 0])/len(after_up_max_2)*100

96.18421487330272

In [60]:
after_days = 5 # 未来5天的
after_up_max = [[] for _ in range(after_days)]  # 未来几天的都
with tqdm(total=len(set(df['id']))) as pbar:
    for j in range(len(ids)-1):
        # 对每一个股票进行遍历
        df2 = df.iloc[ids[j]:ids[j+1],:].copy() # 这里筛选一只股票。
        for i in range(after_days):
            df2['Close{}'.format(i+1)] = df2['Close'].shift(0-(i+1)) # 未来的收盘价移动
        # 默认涨停为0
        df2['up_max'] = 0 
        df2['Close-1'] = df2['Close'].shift(1) # 昨天的收盘价。
        df2.loc[(df2['Close']-df2['Close-1'])/df2['Close-1'] > 0.098, 'up_max'] = 1 # 这一天的是涨停的。
        df2.loc[(df2['High']-df2['Close'])/df2['Close'] > 0.098, 'up_max'] = 1      # 两种涨停
        # 这里要取消开盘价收盘价最高价最低价是一样的情况，这种，我是上不去的。
        df2['up_not'] = 0
        df2.loc[df2['High']==df2['Low'], 'up_not']=1
        df2['up_not'] = df2['up_not'].shift(-1) # 往前一天。
        df2.dropna(axis=0, how='any', inplace=True) # 我这里不需要这个空值，直接删除吧。
        for i in range(after_days):
            # 统计这几天的情况
            df3 = df2.loc[(df2['up_max'] == 1) & (df2['up_not'] == 0), :].copy() # 先筛选
            after_up_max[i].extend(
                ((df3['Close{}'.format(i+1)]-df3['Close'])/df3['Close']*100).tolist()
            )
        pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████| 4275/4275 [02:02<00:00, 34.79it/s]


In [68]:
for i in range(after_days):
    print("往后{}, \t平均涨幅:{}, \t胜率:{:.2f}%, \t差不多涨停的概率:{:.2f}%".format(
        i+1,
        sum(after_up_max[i])/len(after_up_max[i]),
        len([i for i in after_up_max[i] if i > 0])/len(after_up_max[i])*100,
        len([i for i in after_up_max[i] if i > 9])/len(after_up_max[i])*100
    ))

往后1, 	平均涨幅:0.6403400156163194, 	胜率:51.57%, 	差不多涨停的概率:14.44%
往后2, 	平均涨幅:0.5017452837844826, 	胜率:47.28%, 	差不多涨停的概率:14.04%
往后3, 	平均涨幅:0.36440621962879804, 	胜率:46.16%, 	差不多涨停的概率:15.66%
往后4, 	平均涨幅:0.33232944047707824, 	胜率:45.55%, 	差不多涨停的概率:17.12%
往后5, 	平均涨幅:0.16789267715045197, 	胜率:44.49%, 	差不多涨停的概率:17.84%


这个范围真低。

In [62]:
len(after_up_max[0])

81107

In [ ]:
# 我这里判断一下形态，如果形态支持变强，那么最好了。

In [69]:
xingtai = {
    "CDL3INSIDE":talib.CDL3INSIDE, 
    "CDL3STARSINSOUTH":talib.CDL3STARSINSOUTH,
    "CDL3WHITESOLDIERS":talib.CDL3WHITESOLDIERS,
    "CDLBELTHOLD":talib.CDLBELTHOLD,
    "CDLBREAKAWAY":talib.CDLBREAKAWAY,
    "CDLPIERCING":talib.CDLPIERCING,
    "CDLRISEFALL3METHODS":talib.CDLRISEFALL3METHODS,
    "CDLXSIDEGAP3METHODS":talib.CDLXSIDEGAP3METHODS ,
}


after_days = 5 # 未来5天的
after_up_max = {key:[[] for _ in range(after_days)] for key in xingtai}  # 未来几天关于
with tqdm(total=len(set(df['id']))) as pbar:
    for j in range(len(ids)-1):
        # 对每一个股票进行遍历
        df2 = df.iloc[ids[j]:ids[j+1],:].copy() # 这里筛选一只股票。
        for i in range(after_days):
            df2['Close{}'.format(i+1)] = df2['Close'].shift(0-(i+1)) # 未来的收盘价移动
        # 然后这里判断形态
        for key in xingtai:
            df2[key] = xingtai[key](df2['Open'], df2['High'], df2['Low'], df2['Close'])
        # 默认涨停为0
        df2['up_max'] = 0 
        df2['Close-1'] = df2['Close'].shift(1) # 昨天的收盘价。
        df2.loc[(df2['Close']-df2['Close-1'])/df2['Close-1'] > 0.098, 'up_max'] = 1 # 这一天的是涨停的。
        df2.loc[(df2['High']-df2['Close'])/df2['Close'] > 0.098, 'up_max'] = 1      # 两种涨停
        # 这里要取消开盘价收盘价最高价最低价是一样的情况，这种，我是上不去的。
        df2['up_not'] = 0
        df2.loc[df2['High']==df2['Low'], 'up_not']=1
        df2['up_not'] = df2['up_not'].shift(-1) # 往前一天。
        df2.dropna(axis=0, how='any', inplace=True) # 我这里不需要这个空值，直接删除吧。
        for i in range(after_days):
            # 统计这几天的情况
            for key in xingtai:
                # 这里还要判断形态
                df3 = df2.loc[(df2['up_max'] == 1) & (df2['up_not'] == 0) & (df2[key] > 0), :].copy() # 先筛选
                after_up_max[key][i].extend(
                    ((df3['Close{}'.format(i+1)]-df3['Close'])/df3['Close']*100).tolist()
                )

        pbar.update(1)

100%|██████████████████████████████████████████████████████████████████████████████| 4275/4275 [09:13<00:00,  7.72it/s]


In [78]:
for key in xingtai:
    print("{} - {} 次".format(key, len(after_up_max[key][0])))
    if len(after_up_max[key][0]) > 0:
        for i in range(after_days):
            print("{}:往后{}, \t平均涨幅:{}, \t胜率:{:.2f}%, \t差不多涨停的概率:{:.2f}%".format(
                key,
                i+1,
                sum(after_up_max[key][i])/len(after_up_max[key][i]),
                len([i for i in after_up_max[key][i] if i > 0])/len(after_up_max[key][i])*100,
                len([i for i in after_up_max[key][i] if i > 9])/len(after_up_max[key][i])*100
            ))
    print("*"*120)

CDL3INSIDE - 1332 次
CDL3INSIDE:往后1, 	平均涨幅:1.3952736147473919, 	胜率:56.01%, 	差不多涨停的概率:13.21%
CDL3INSIDE:往后2, 	平均涨幅:1.7042643965655186, 	胜率:54.35%, 	差不多涨停的概率:13.36%
CDL3INSIDE:往后3, 	平均涨幅:1.5796566650033255, 	胜率:53.30%, 	差不多涨停的概率:16.29%
CDL3INSIDE:往后4, 	平均涨幅:1.4819363747787697, 	胜率:51.58%, 	差不多涨停的概率:17.42%
CDL3INSIDE:往后5, 	平均涨幅:1.0470589982616634, 	胜率:49.10%, 	差不多涨停的概率:17.42%
************************************************************************************************************************
CDL3STARSINSOUTH - 0 次
************************************************************************************************************************
CDL3WHITESOLDIERS - 269 次
CDL3WHITESOLDIERS:往后1, 	平均涨幅:1.1187481091526534, 	胜率:53.53%, 	差不多涨停的概率:24.16%
CDL3WHITESOLDIERS:往后2, 	平均涨幅:0.40319611552806184, 	胜率:43.49%, 	差不多涨停的概率:16.36%
CDL3WHITESOLDIERS:往后3, 	平均涨幅:0.13171363756139207, 	胜率:44.98%, 	差不多涨停的概率:16.73%
CDL3WHITESOLDIERS:往后4, 	平均涨幅:0.41318099623749016, 	胜率:41.64%, 	差不多涨停的概率:20.45%
CDL3WHITESOLDIERS:往

这里解释一下，  
CDLRISEFALL3METHODS ： 上升/下降三法，日K线模式，以上升三法为例，上涨趋势中，第一日长阳线，中间三日价格在第一日范围内小幅震荡，第五日长阳线，收盘价高于第一日收盘价，预示股价上升   
CDLPIERCING ： 刺透形态，两日K线模式，下跌趋势中，第一日阴线，第二日收盘价低于前一日最低价，收盘价处在第一日实体上部，预示着底部反转